# Let's Train GPT-2



In [1]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    !git clone https://github.com/novastar53/jaxpt
    !cd jaxpt && git checkout dev && git pull
    !pip install tiktoken --quiet
    !pip uninstall -y tensorflow

fatal: destination path 'jaxpt' already exists and is not an empty directory.
Already on 'dev'
Your branch is up to date with 'origin/dev'.
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 9 (delta 4), reused 9 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (9/9), 26.17 KiB | 893.00 KiB/s, done.
From https://github.com/novastar53/jaxpt
   8420412..1b9a060  dev        -> origin/dev
Updating 8420412..1b9a060
Fast-forward
 .../datasets/panchatantra-ryder/processed/panchatantra_train_0.npy  | Bin 1304808 -> 1174336 bytes
 .../datasets/panchatantra-ryder/processed/panchatantra_valid_1.npy  | Bin 0 -> 130600 bytes
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 src/jaxpt/datasets/panchatantra-ryder/processed/panchatantra_valid_1.npy


In [2]:
from pathlib import Path
import sys

if is_colab():
    jaxpt_dir = str(Path().absolute() / "jaxpt" / "src" )
else:
    jaxpt_dir = str(Path().absolute().parent / "src" )

sys.path.append(jaxpt_dir)
print(jaxpt_dir)

/content/jaxpt/src


In [3]:
import jax
import optax
import jax.numpy as jnp
import numpy as np
from flax import nnx
import tiktoken

from jaxpt.dataloaders import DataLoader
from jaxpt.models import GPT2, GPTConfig
from jaxpt.train import train_step, loss_fn, compute_global_norm
from jaxpt.infer import generate

### Configure compute

In [4]:
import os

# Hardware setup
print("JAX version:", jax.__version__)
devices = jax.devices()
num_devices = len(devices)
print("Available devices:", num_devices)

jax.config.update("jax_platform_name", "gpu") # Make sure we're using the GPU
#jax.config.update("jax_enable_x64", True) # Make sure the highest precision is enabled in case we need
jax.config.update("jax_default_matmul_precision", "bfloat16") # Set the default precision for matrix multiplication

os.environ["NVIDIA_TF32_OVERRIDE"] = "1"
#os.environ["JAX_ENABLE_X64"] = "False"

def list_tpu_memory():
    devices = jax.devices()
    for device in devices:
        if 'TPU' in str(device.device_kind):
            print(f"Device: {device}, Memory: {device.memory_stats()['bytes_limit']/(1024*1024)},  Used: {device.memory_stats()['bytes_in_use']/(1024*1024)}")

list_tpu_memory()

print("Using device:", jax.default_backend())  # Should print 'gpu'

#A = jnp.array(np.random.normal(size=(4096, 4096)), dtype=jnp.float32) # Makes sure the matmul is fast

#%timeit (A@A).block_until_ready()

JAX version: 0.4.33
Available devices: 8
Device: TPU_0(process=0,(0,0,0,0)), Memory: 7661.984375,  Used: 0.015625
Device: TPU_1(process=0,(0,0,0,1)), Memory: 7661.984375,  Used: 0.015625
Device: TPU_2(process=0,(1,0,0,0)), Memory: 7661.984375,  Used: 0.015625
Device: TPU_3(process=0,(1,0,0,1)), Memory: 7661.984375,  Used: 0.015625
Device: TPU_4(process=0,(0,1,0,0)), Memory: 7661.984375,  Used: 0.015625
Device: TPU_5(process=0,(0,1,0,1)), Memory: 7661.984375,  Used: 0.015625
Device: TPU_6(process=0,(1,1,0,0)), Memory: 7661.984375,  Used: 0.015625
Device: TPU_7(process=0,(1,1,0,1)), Memory: 7661.984375,  Used: 0.015625
Using device: tpu


### Initialize the GPT-2 model and perform a sanity check

In [5]:
from functools import partial

"""
+--------------+---------+--------+------+
| Model        | Layers  | Heads  | Embd |
+--------------+---------+--------+------+
| gpt2-medium  | 24      | 16     | 1024 |
| gpt2-large   | 36      | 20     | 1280 |
| gpt2-xl      | 48      | 25     | 1600 |
+--------------+---------+--------+------+
"""

key = jax.random.PRNGKey(0)
rngs = nnx.Rngs({"dataloader": key, "dropout": key, "params": key, "generate": key})
config = GPTConfig(dtype=jnp.float32)
m = GPT2(config, rngs)

def generate_completions():
  m.eval()
  num_completions = 5
  max_length = 20
  generate_completion = partial(generate, m, max_length=max_length)
  prefix = "The clever jackal"
  enc = tiktoken.get_encoding('gpt2')
  tokens = enc.encode(prefix)
  tokens = jnp.array(tokens, dtype=jnp.int32)
  tokens = jnp.expand_dims(tokens, axis=0)
  x = jnp.tile(tokens, (num_completions, 1))


  x = generate_completion(x=x) # Make sure you can do a forward pass
  for i in range(num_completions):
      tokens = x[i, :max_length].tolist()
      decoded = enc.decode(tokens)
      print(">", decoded)

#generate_completions()

### Training setup

In [6]:
num_tokens_per_batch = 2**16 # 2**19, 0.5 million as per the GPT 3.5 paper
mB, T = 16 , 256
grad_accumulation_steps = num_tokens_per_batch // (mB * T * num_devices) # Number of steps over which to average the gradient
print(f"tokens/batch: {num_tokens_per_batch:,}")
print(f"block size: {T}")
print(f"sub-batch size: {mB}")
print(f"no. gradient accumulation steps: {grad_accumulation_steps}")
print(f"effective batch size per device: ", grad_accumulation_steps * mB)
print(f"effective batch size: {grad_accumulation_steps * mB * num_devices}")


max_steps = 100
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
eval_interval = 10

print(f"max steps: {max_steps}")

# Set up the optimizer
def warmup_with_cosine_decay_schedule(step):

    warmup_lr = max_lr * (step + 1) / warmup_steps

    coeff = 0.5 * (1 + jnp.cos(jnp.pi * (step - warmup_steps) / (max_steps - warmup_steps)))
    cosine_lr =  min_lr + coeff * (max_lr - min_lr)

    return jnp.where(step < warmup_steps,
                     warmup_lr,
                     jnp.where(step < max_steps, cosine_lr, min_lr))

# Generate a weight decay mask
# First split the model into params and variables
graphdef, params, variables = nnx.split(m, nnx.Param, nnx.Variable)
# Then create a mask for the weight decay params
weight_decay_mask = jax.tree_util.tree_map(lambda x: len(x.shape) > 1, params)

def f(x, y):
    if x:
        return y.size
    return 0

weight_decay_params = jax.tree_util.tree_map(f, weight_decay_mask, params)
weight_decay_param_count = jax.tree_util.tree_reduce(lambda x, y: x + y, weight_decay_params, 0)
print(f"weight decay param count: {weight_decay_param_count:,}")

max_grad_norm = 1.0  # Clip gradients to this norm

tx = optax.chain(
    optax.clip_by_global_norm(max_grad_norm),
    optax.adamw(warmup_with_cosine_decay_schedule, b1=0.9, b2=0.95, weight_decay=0.1, mask=weight_decay_mask)
)
optimizer = nnx.Optimizer(m, tx)

tokens/batch: 65,536
block size: 256
sub-batch size: 16
no. gradient accumulation steps: 2
effective batch size per device:  32
effective batch size: 256
max steps: 100
weight decay param count: 124,354,560


In [7]:

def print_separator(title=None):
    width = 80
    border = "═" * width
    if title:
        padding = "═" * ((width - len(title) - 2) // 2)
        print(f"╔{border}╗")
        print(f"║{padding} {title} {padding}║")
        print(f"╚{border}╝")
    else:
        print(f"╔{border}╗")
        print(f"╚{border}╝")

# Usage


if is_colab():
    dataset_path = Path().absolute() / "jaxpt" / "src" / "jaxpt" / "datasets" / "panchatantra-ryder" / "processed"
else:
    dataset_path = Path().absolute().parent / "src"/ "jaxpt" / "datasets" / "panchatantra-ryder" / "processed"


def validate(m):
  eval_dl = DataLoader(dirpath=dataset_path, batch_size=mB, block_size=T, device_rank=1, label="valid")
  valid_loss = 0.0
  eval_steps = 10
  for i in range(eval_steps):
    batch, targets = eval_dl()
    batch = np.squeeze(batch)
    targets = np.squeeze(targets)
    loss = loss_fn(m, batch, targets)
    valid_loss += loss
  valid_loss /= eval_steps
  print(f"valid loss: {valid_loss:0.4f}")

def evaluate(m):
  print_separator("Evaluate")
  m.eval()
  generate_completions()
  print_separator()
  validate(m)
  print_separator()
  m.train()

In [13]:
class DataLoader:
    def __init__(self, dirpath: str, batch_size: int, block_size: int, device_rank: int, label: str | None=None, quiet: bool=False):
        self.dirpath = dirpath
        self.enc = tiktoken.get_encoding("gpt2")
        self.eot = self.enc._special_tokens['<|endoftext|>']

        self.B = batch_size
        self.T = block_size
        self.D = device_rank

        self.shards = os.listdir(dirpath)
        if label is not None:
            self.shards = [shard for shard in self.shards if label in shard]

        self.cur_shard = 0
        self.pos = 0
        self.shard = self.__load_shard()
        self.shard_size = len(self.shard)

        if not quiet:
            print(f"""dataloader initialized:
------------------------
f"label:          {label}
f"shards:         {len(self.shards):,}
f"shard size:     {self.shard_size:,}
f"batch size:     {self.B}
f"block size:     {self.T}
f"device rank:    {self.D}
------------------------""")

    def __len__(self):
        return len(self.shards) * self.shard_size

    def __load_shard(self):
        shard = self.shards[self.cur_shard]
        tokens = np.load(os.path.join(self.dirpath, shard))
        if type(tokens) is not np.ndarray:
            tokens = tokens["arr_0"]
        self.shard_size = len(tokens)
        return tokens

    def __call__(self):
        # preallocate the  buffer
        buf_size = self.B * self.T * self.D + 1
        buf = np.zeros((buf_size,), dtype=np.uint16)

        # if the shard has enough tokens remaining
        if self.pos + buf_size < self.shard_size:
            buf[:] = self.shard[self.pos:self.pos + buf_size]
            self.pos += buf_size
        else:
            # load the buffer in part
            buf_prefix_size = buf_size - (self.shard_size - self.pos)
            print(buf_prefix_size, self.shard_size)
            buf[:buf_prefix_size] = self.shard[self.pos:self.pos + buf_prefix_size]

             # load the next shard
            if self.cur_shard < len(self.shards) - 1:
                self.cur_shard += 1
            else:
                self.cur_shard = 0
            self.shard = self.__load_shard()
            self.pos = 0

            # load the remainder of the buffer
            buf[buf_prefix_size:] = self.shard[:buf_size - buf_prefix_size]
            self.pos = (buf_size - buf_prefix_size)

        X = buf[:-1].reshape((self.D, self.B, self.T))
        Y = buf[1:].reshape((self.D, self.B, self.T))

        return jnp.array(X), jnp.array(Y)

In [14]:
train_dl = DataLoader(dirpath=dataset_path, batch_size=mB, block_size=T, device_rank=num_devices, label="train")

dataloader initialized:
------------------------
f"label:          train
f"shards:         1
f"shard size:     146,776
f"batch size:     16
f"block size:     256
f"device rank:    8
------------------------


In [ ]:
%%time
from time import time
from IPython.display import clear_output
from functools import partial

#m = GPT2(config, rngs)
# optimizer = nnx.Optimizer(m, tx)
list_tpu_memory()
evaluate(m)
m.train()

try:
  for step in range(max_steps):
    start = time()
    accum_grad = None
    accum_loss = 0.0
    for sub_step in range(grad_accumulation_steps):
      batch, targets = train_dl()
      loss, grads = train_step(m, batch, targets)
      # retrieve a single value from the all-reduce op
      loss = loss[0]
      grads = jax.tree_util.tree_map(lambda x: x[0, ...], grads)
      if accum_grad is None:
        accum_grad = jax.tree_util.tree_map(jnp.zeros_like, grads)
      # accumulate the gradients and loss
      accum_grad = jax.tree_util.tree_map(lambda x, y: x + y, accum_grad, grads)
      accum_loss = accum_loss + loss
       #jax.block_until_ready(accum_grad)

    # average the gradients across accumulation steps
    accum_grad = jax.tree_util.tree_map(lambda x: x / grad_accumulation_steps, accum_grad)
    accum_loss = accum_loss / grad_accumulation_steps
    # update the model with the averaged gradients
    optimizer.update(accum_grad)

    iter_time = (time() - start)

    # compute stats
    lr = warmup_with_cosine_decay_schedule(step)
    norm = compute_global_norm(accum_grad)
    sub_step_time = iter_time / grad_accumulation_steps
    tokens_per_sec = num_devices * mB * T * grad_accumulation_steps / iter_time
    tokens_processed = (step+1) * num_devices * grad_accumulation_steps * mB * T

    # print the stats
    #clear_output(wait=True)
    print(f"{step} | lr: {lr:0.2e} | loss: {accum_loss:0.4f} | norm: {norm:0.2f} | time: {sub_step_time*1000:0.2f}ms | tokens processed: {tokens_processed:,} | tok/sec: {tokens_per_sec:0.2f}")

    if step % eval_interval == 1:
      evaluate(m)
except KeyboardInterrupt:
    print("Received KeyboardInterrupt. Exiting...")
evaluate(m)

Device: TPU_0(process=0,(0,0,0,0)), Memory: 7661.984375,  Used: 2482.25
Device: TPU_1(process=0,(0,0,0,1)), Memory: 7661.984375,  Used: 48.140625
Device: TPU_2(process=0,(1,0,0,0)), Memory: 7661.984375,  Used: 48.140625
Device: TPU_3(process=0,(1,0,0,1)), Memory: 7661.984375,  Used: 48.140625
Device: TPU_4(process=0,(0,1,0,0)), Memory: 7661.984375,  Used: 48.140625
Device: TPU_5(process=0,(0,1,0,1)), Memory: 7661.984375,  Used: 48.140625
Device: TPU_6(process=0,(1,1,0,0)), Memory: 7661.984375,  Used: 48.140625
Device: TPU_7(process=0,(1,1,0,1)), Memory: 7661.984375,  Used: 48.140625
╔════════════════════════════════════════════════════════════════════════════════╗
║═══════════════════════════════════ Evaluate ═══════════════════════════════════║
╚════════════════════════════════════════════════════════════════════════════════╝
> The clever jackal,ped Razer ME mailbox the venturebots Erit Wings enzymes. sunglassesunte pos mailbox
> The clever jackalauntletollower Newman Newman " wind SH